In [1]:
import numpy as np
import QuantLib as ql
import csv

In [2]:
today = ql.Date(1,1,2022)
ql.Settings.instance().setEvaluationDate(today)
day_counter = ql.Actual365Fixed()

In [3]:
def fdm(strike, forward, expiry_time, alpha, beta, nu, rho, t_grid=20, f_grid=200, x_grid=25, scaling_factor=1.0, eps=.0001):
    maturity_date = today + ql.Period(int(365*expiry_time), ql.Days)
    r_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.0, day_counter))
    vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(0, ql.NullCalendar(), 0.0, day_counter))
    bs_process = ql.GeneralizedBlackScholesProcess(ql.QuoteHandle(ql.SimpleQuote(forward)), r_ts, r_ts, vol_ts)
    option = ql.VanillaOption(ql.PlainVanillaPayoff(ql.Option.Call, strike), ql.EuropeanExercise(maturity_date))
    option.setPricingEngine(ql.FdSabrVanillaEngine(forward, alpha, beta, nu, rho, r_ts, t_grid, f_grid, x_grid, 0, scaling_factor, eps))
    return option.impliedVolatility(option.NPV(), bs_process, 1.0e-4, 1000, 1.0e-5, 1.0)

In [ ]:
file_name = 'fdm_hagan_vol_train.csv'
strikes = [0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
tenors = [0.5, 1, 1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10]
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
betas = [0.001, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
nus = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
rhos = [-0.1, -0.075, -0.05, -0.025, 0, 0.025, 0.05, 0.075, 0.1]

with open(file_name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Strike", "Tenor", "Alpha", "Beta", "Nu", "Rho", "FDM_vol", "Hagan_vol"])

for strike in strikes:
    for tenor in tenors:
        for alpha in alphas:
            with open(file_name, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                for beta in betas:
                    for nu in nus:
                        for rho in rhos:
                            try:
                                fdm_vol = fdm(strike, 1, tenor, alpha, beta, nu, rho, 40, 200, 25, eps=.00001)
                                if fdm_vol != 1.0e-5:
                                    writer.writerow([
                                        strike,
                                        tenor,
                                        alpha,
                                        beta,
                                        nu,
                                        rho,
                                        fdm_vol,
                                        ql.sabrVolatility(strike, 1, tenor, alpha, beta, nu, rho)
                                    ])
                            except:
                                pass


In [ ]:
with open('fdm_hagan_vol_test.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Strike", "Tenor", "Alpha", "Beta", "Nu", "Rho", "FDM_vol", "Hagan_vol"])

    count = 0
    while count < 50000:
        strike = np.random.uniform(0.8, 1.3)
        tenor = np.random.uniform(0.5, 10)
        alpha = np.random.uniform(0.1, 0.9)
        beta = np.random.uniform(0.001, 0.5)
        nu = np.random.uniform(0.1, 0.5)
        rho = np.random.uniform(-0.1, 0.1)
        try:
            fdm_vol = fdm(strike, 1, tenor, alpha, beta, nu, rho, 40, 200, 25, eps=.00001)
            if fdm_vol != 1.0e-5:
                writer.writerow([
                    strike,
                    tenor,
                    alpha,
                    beta,
                    nu,
                    rho,
                    fdm_vol,
                    ql.sabrVolatility(strike, 1, tenor, alpha, beta, nu, rho)
                ])
                count += 1
        except:
            pass
